## Nonnegative Matrix Factorization

In [59]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

NMF is a method to factorize a nonnegative matrix $X_{m \times n}$ to nonnegative matrix factors $W_{m \times r}$ and $H_{r \times n}$ such that $X \approx WH$. Here, $r$ is a parameter to be determined.

### Iterative Algorithm for NMF
An iterative algorithm for NMF as proposed in [[1]](#references).

In [24]:
def obj_func(orig, approx):
    """
    Compute the objective function value for NMF given the original
    and approximate matrices.
    
    Parameters
    ----------
    orig : numpy.array
        Original matrix to factorize.
    approx : numpy.array
        Current approximate matrix.
        
    Returns
    -------
    obj_value : float
        Current objective function value.
    """
    divergence_matrix = orig*np.log(approx) - approx
    return np.sum(divergence_matrix)

def nmf(X, r):
    """
    Given a nonnegative matrix X of shape m x n, iteratively
    compute a nonnegative matrix factorization X = WH, and
    return the factors W and H.
    
    The returned factors have shapes (m, r) and (r, n),
    respectively.
    
    Parameters
    ----------
    X : numpy.array
        m x n nonnegative matrix.
    r : int
        Number of columns of factor W.
        
    Returns
    -------
    W : numpy.array
        m x r nonnegative matrix.
    H : numpy.array
        r x n nonnegative matrix.
    """
    m, n = X.shape
    W = np.random.rand(m, r)
    H = np.random.rand(r, n)
    prev_objs = np.zeros(100)
    prev_index = 0
    while True:
        XA = W@H  
        
        # Convergence check
        curr_obj = obj_func(X, XA)
        if abs(curr_obj - np.mean(prev_objs)) < 1e-6:
            break
        else:
            prev_objs[prev_index] = curr_obj
            prev_index = (prev_index + 1)%100
            
        # Update each element of W
        for i in range(m):
            for a in range(r):
                coeff = np.sum(X[i, :]/XA[i, :]*H[a, :])
                W[i, a] *= coeff
        
        # Normalize columns of W st they sum to 1
        for a in range(r):
            W[:, a] /= np.sum(W[:, a])
        
        # Update each element of H
        for a in range(r):
            for mu in range(n):
                coeff = np.sum(W[:, a]*X[:, mu]/XA[:, mu])
                H[a, mu] *= coeff
                
    return W, H

In [71]:
X = np.random.randint(low=0, high=100, size=(5, 50))
W, H = nmf(X, 3)

## Bayesian Inference For NMF

### $r = 1$ Case
In this case, we factorize $X$ into row and column vectors $W$ and $H$ whose outer product is the approximation $\tilde{X}$.

$$
\begin{align}
    \begin{pmatrix}
      x_{11} & \dots & x_{1n} \\
      \vdots & \ddots & \vdots \\
      x_{m1} & \dots & x_{mn}
    \end{pmatrix}
    \approx
    \begin{pmatrix}
      w_1 \\
      \vdots \\
      w_m
    \end{pmatrix}
    \begin{pmatrix}
      h_1 & \dots & h_n
    \end{pmatrix}
\end{align}
$$

In [ ]:
def rank_one_nmf(X):
    """
    Given a nonnegative matrix X, construct nonnegative vectors
    w and h whose outer product approximates X.
    """
    m, n = X.shape
    w = X[:, 1]
    h = np.ones(n)
    for i in range(1, n):
        # find h_i such that h_i*w is as close to X[i] as possible
        
    return w, h

In [11]:
w = np.array([5, 3, 7])
h = np.array([0.1, 2, 3, 0.7])
X = np.outer(w, h)
print(X)

[[  0.5  10.   15.    3.5]
 [  0.3   6.    9.    2.1]
 [  0.7  14.   21.    4.9]]


<a id='references'></a>
## References
1. D. Lee and H. S. Seung, “Learning the parts of objects with nonnegative matrix factorization,” Nature, vol. 401, pp. 788– 791, 1999.